In [ ]:
import sys, os

sys.path.append("../bright_objects_masks")
import generate_masks
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

We didn't specify any density_ratio in the config file. This will be calculated in the init of Masks class as well as the radius value.

In [ ]:
curr_dir = os.getcwd()
mask_gen = generate_masks.Masks(
    config_file=curr_dir + "/../config/config_examples.yaml"
)

Now we create the healsparse map.

In [ ]:
mask = mask_gen.create_healsparse_masks()

Save it

In [ ]:
mask_gen.write_heaslparse_mask(mask)

In [ ]:
outpath = mask_gen.outpath

We read it back to show how to do but it is not mandatory as we computed it in upper cells.

In [ ]:
import healsparse as hsp

mask = hsp.HealSparseMap.read(outpath + "bo_masks.hs")

The best way to vizualize masks is to use the skyproj package.

In [ ]:
import skyproj

fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax)
im, lon_raster, lat_raster, values_raster = sp.draw_hspmap(
    mask, vmin=0, vmax=1, lon_range=[59, 62], lat_range=[-37, -35], cmap="Dark2"
)
# for star in stars_dc2[(stars_dc2['ra']>60)&(stars_dc2['ra']<60.5)&(stars_dc2['dec']<-36)&(stars_dc2['dec']>-36.4)]:
#    sp.draw_polygon([star['ra']+0.0001, star['ra']-0.0001], [star['dec']+0.0001, star['dec']-0.0001], alpha=.7, facecolor = 'red', edgecolor='red')
plt.annotate(
    "Masks vs stars",
    xy=(0.5, 1.03),
    xycoords="axes fraction",
    xytext=(0, 10),
    textcoords="offset points",
    ha="center",
    va="bottom",
    fontsize=15,
)
plt.colorbar()
plt.show()

We can either compare to some footprint

In [ ]:
import GCRCatalogs
from GCRCatalogs.helpers.tract_catalogs import tract_filter
from astropy.table import Table
import numpy as np

name = "dc2_object_run2.2i_dr6_v2_with_addons_v2"
catalog = GCRCatalogs.load_catalog(name)
dc2_table = Table(
    catalog.get_quantities(
        [
            "ra",
            "dec",
            "extendedness",
            "mag_i_cModel",
            "ID",
            "id",
            "id_truth",
            "cosmodc2_id_truth",
            "truth_type",
            "mag_i_truth",
            "clean",
            "blendedness",
        ],
        native_filters=[tract_filter(3830)],
    )
)
matteo_gals = dc2_table[
    (dc2_table["extendedness"] == 1)
    & (dc2_table["clean"] == True)
    & (np.isfinite(dc2_table["mag_i_cModel"]))
]
bright_stars = dc2_table[
    (dc2_table["extendedness"] != 1)
    & (dc2_table["clean"] == False)
    & (np.isnan(dc2_table["mag_i_cModel"]))
    & (dc2_table["mag_i_truth"] < 12)
]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax)
im, lon_raster, lat_raster, values_raster = sp.draw_hspmap(
    mask,
    vmin=0,
    vmax=1,
    lon_range=[max(matteo_gals["ra"]), min(matteo_gals["ra"])],
    lat_range=[min(matteo_gals["dec"]), max(matteo_gals["dec"])],
    cmap="Dark2",
)
sp.plot(matteo_gals["ra"], matteo_gals["dec"], "r.", markersize=0.05, alpha=0.1)
sp.plot(bright_stars["ra"], bright_stars["dec"], "b.", markersize=2, alpha=1)
plt.annotate(
    "Masks vs stars",
    xy=(0.5, 1.03),
    xycoords="axes fraction",
    xytext=(0, 10),
    textcoords="offset points",
    ha="center",
    va="bottom",
    fontsize=15,
)
plt.colorbar()
plt.show()

In [ ]:
galaxy_sample_r = matteo_gals[
    (matteo_gals["ra"] >= 60.16)
    & (matteo_gals["ra"] <= 60.45)
    & (matteo_gals["dec"] <= -36.11)
    & (matteo_gals["dec"] >= -36.34)
]
bright_stars_r = bright_stars[
    (bright_stars["ra"] >= 60.16)
    & (bright_stars["ra"] <= 60.45)
    & (bright_stars["dec"] <= -36.11)
    & (bright_stars["dec"] >= -36.34)
]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.McBrydeSkyproj(ax=ax)
im, lon_raster, lat_raster, values_raster = sp.draw_hspmap(
    mask,
    vmin=0,
    vmax=1,
    lon_range=[60.45, 60.16],
    lat_range=[-36.34, -36.11],
    cmap="Dark2",
)
sp.plot(galaxy_sample_r["ra"], galaxy_sample_r["dec"], "r.", markersize=1, alpha=0.5)
sp.plot(bright_stars_r["ra"], bright_stars_r["dec"], "b.", markersize=2, alpha=1)
sp.plot(
    bright_stars_r["ra"][bright_stars_r["mag_i_truth"] < 10],
    bright_stars_r["dec"][bright_stars_r["mag_i_truth"] < 10],
    "c.",
    markersize=2,
    alpha=1,
)
plt.annotate(
    "Masks vs stars",
    xy=(0.5, 1.03),
    xycoords="axes fraction",
    xytext=(0, 10),
    textcoords="offset points",
    ha="center",
    va="bottom",
    fontsize=15,
)
plt.colorbar()
plt.show()